### open-ko-llm_LeaderBoard
업스테이지, NIA 의 https://huggingface.co/spaces/upstage/open-ko-llm-leaderboard 참고

In [1]:
!nvidia-smi

Wed Dec  4 15:12:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 566.03       CUDA Version: 12.7     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060        On  | 00000000:07:00.0 Off |                  N/A |
|  0%   37C    P8              12W / 180W |    574MiB / 12288MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# !pip install transformers bitsandbytes accelerate

### linkbricks 사의 모델 이용
- Benchmark (Open Ko LLM Leader Board Season 2 : No. 1)
- Model : Saxo/Linkbricks-Horizon-AI-Korean-Mistral-Nemo-sft-dpo-12B
- Average : 51.37
- Ko-GPQA : 25.25
- Ko-Winogrande : 68.27
- Ko-GSM8k : 70.96
- Ko-EQ Bench : 50.25
- Ko-IFEval : 49.84
- KorNAT-CKA : 34.59
- KorNAT-SVA : 48.42
- Ko-Harmlessness : 65.66
- Ko-Helpfulness : 49.12

### 양자화 로딩 과정

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
model_name = "Saxo/Linkbricks-Horizon-AI-Korean-Mistral-Nemo-sft-dpo-12B"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

2024-12-04 15:14:54.575413: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733292894.593866    9667 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733292894.599780    9667 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-04 15:14:54.620360: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


model-00001-of-00005.safetensors:  13%|#3        | 640M/4.87G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/181k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.26M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [59]:
print(f"""bnb_config 에 따라서 아래와 같이 4비트로 계산한다
{"-"*100}
{model.model.layers[0-39].self_attn.q_proj}
{model.model.layers[0-39].self_attn.k_proj}
{model.model.layers[0-39].self_attn.v_proj}
""")

bnb_config 에 따라서 아래와 같이 4비트로 계산한다
----------------------------------------------------------------------------------------------------
Linear4bit(in_features=5120, out_features=4096, bias=False)
Linear4bit(in_features=5120, out_features=1024, bias=False)
Linear4bit(in_features=5120, out_features=1024, bias=False)



In [68]:
# 로컬 모델 테스트
messages = [
    {"role": "user", "content": "오늘 날씨는 어떤가요?"},
]
model_inputs = tokenizer.apply_chat_template(messages,return_tensors="pt").to("cuda")

result = model.generate(model_inputs,
                        max_new_tokens=100,
                        do_sample=True
    )

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [69]:
print(tokenizer.decode(result[0]).split("[/INST]")[-1])

현재 사용중인 지역 기반의 기상 정보를 기반으로 오늘의 날씨 정보를 제공할 수 있습니다. Jérusalem(예시)로 설정되었습니다. 지금 Jerusalem에서의 날씨는 흐려서 소랑한데 최고 기온은 20도이고 최저 기온은 12도입니다. 바람은 약 5km/h 정도이고 비는 없으며 습도는 70%입니다. 오늘의 날씨는 대체로 상쾌하고 관상적인 편


### 랭체인 구성하기

In [76]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import pipeline
from langchain.chains import LLMChain

text_gen_pipe = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.3,
    return_full_text=True,
    max_new_tokens=300
)

prompt_template = """
### [INST]
instruction: Answer the question based on your economic knowledge.
here is context to help:

{context}

### QUESTION:
{question}

[/INST]
"""

In [78]:
mistral_llm = HuggingFacePipeline(pipeline=text_gen_pipe)

prompt = PromptTemplate(
    input_variables=["context","question"],
    template=prompt_template
)

llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

/tmp/ipykernel_9667/4008923309.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)


In [80]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.schema.runnable import RunnablePassthrough

In [94]:
loader = PyPDFLoader("./docs/doc_1.pdf")
docs = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(docs)

from langchain.embeddings import HuggingFaceEmbeddings

embeding_model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {"normalize_ebeddings":True} # 거리 계산(코사인 유사도 등) 을 위해 정규화 과정
hf = HuggingFaceEmbeddings(
    model_name=embeding_model_name,
    encode_kwargs=encode_kwargs
)

db = FAISS.from_documents(texts, hf)

# db 를 retriever로 이용할 것(유사성 기반으로 검색을 할 것이다)
retriever = db.as_retriever(search_type="similarity",
                            search_kwargs={'k':1})

In [95]:
# "context" : retriever 검색을 이용하여 저장
# "question" : 입력 그대로 통과
# | : 입력 dict 를 뒤에 llm_chain 으로 전달한다.
rag_chain = (
    {"context":retriever, "question":RunnablePassthrough()} | llm_chain
)

In [108]:
result = rag_chain.invoke("도쿄에서 아무도 어떻게 할 수 없다고?")

/home/tommy/anaconda3/envs/transformer/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [109]:
# 참고 도큐멘트
result["context"]


[Document(metadata={'source': './docs/doc_1.pdf', 'page': 0}, page_content="(4, 3, 2, 1) \nNo one sleep in Tokyo \nAll right crossing the line \nNo one quit the radio \nTokyo is on fire \nEven if you say: I have been the world wide \nI'll take you where surely you have never been \nAll night in the fight I'm okay, come on \nCome on \nHey do you feel the night is breathable \nLook at this town which is unbelievable \nNo other places like that in the world, world, world \n \n \n(1, 2, 3, 4) \nNo one sleep in Tokyo \nAll night crossing the line \nNo one quit the radio \nTokyo is on fire")]

In [110]:
print(result["text"])


### [INST]
instruction: Answer the question based on your economic knowledge.
here is context to help:

[Document(metadata={'source': './docs/doc_1.pdf', 'page': 0}, page_content="(4, 3, 2, 1) \nNo one sleep in Tokyo \nAll right crossing the line \nNo one quit the radio \nTokyo is on fire \nEven if you say: I have been the world wide \nI'll take you where surely you have never been \nAll night in the fight I'm okay, come on \nCome on \nHey do you feel the night is breathable \nLook at this town which is unbelievable \nNo other places like that in the world, world, world \n \n \n(1, 2, 3, 4) \nNo one sleep in Tokyo \nAll night crossing the line \nNo one quit the radio \nTokyo is on fire")]

### QUESTION:
도쿄에서 아무도 어떻게 할 수 없다고?

[/INST]
ANSWER:
"도쿄에서 아무도 잠을 자지 않는다"는 구절은 도쿄에서 아무도 잠을 자지 않는다는 것을 의미합니다. 즉, 도쿄는 밤새도록 활동적인 도시라는 것을 암시합니다.


### 리소스 사용량 확인 - 3060gpu using : (11gb/12gb)

In [111]:
!nvidia-smi

Wed Dec  4 16:22:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 566.03       CUDA Version: 12.7     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060        On  | 00000000:07:00.0 Off |                  N/A |
|  0%   45C    P8              12W / 180W |  11042MiB / 12288MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--